### Monday, January 29, 2024

conda activate langchain2

Trying this notebook using the LMStudio server from the 'langchain2' conda environment.

NOTE: In LMStudio, setting n_gpu_layers to -1 will attempt to load THE ENTIRE MODEL to the gpu, and it this works, there is a huge performance gain. Loading anything to the CPU drastically degrades performance.

### Friday, January 26, 2024

Trying this notebook using the LMStudio server from the 'langchain' conda environment.

Nice! This all runs in one pass!

### Wednesday, November 22, 2023

[Prompt Templates for GPT 3.5 and other LLMs - LangChain #2](https://www.youtube.com/watch?v=RflBcK0oDH0&list=PLIUOU7oqGTLieV9uTIFMm6_4PXg-hlN6F&index=2)

[Prompt Engineering and LLMs with Langchain](https://www.pinecone.io/learn/series/langchain/langchain-prompt-templates/)

Start : OpenAI Usage = $1.64

End: 

### Monday, November 20, 2023

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb

This all runs in one pass.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/01-langchain-prompt-templates.ipynb)

# Prompt Engineering

In this notebook we'll explore the fundamentals of prompt engineering. We'll start by installing library prerequisites.

In [1]:
# !pip install langchain openai

## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text. For a model generating Python code we may put `import ` (as most Python scripts begin with a library `import`), or a chatbot may begin with `Chatbot: ` (assuming we format the chatbot script as lines of interchanging text between `User` and `Chatbot`).

Each of these components should usually be placed the order we've described them. We start with instructions, provide context (if needed), then add the user input, and finally end with the output indicator.

In [1]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

In this example we have:

```
Instructions

Context

Question (user input)

Output indicator ("Answer: ")
```

Let's try sending this to a GPT-3 model. We will use the LangChain library but you can also use the `openai` library directly. In both cases, you will need [an OpenAI API key](https://beta.openai.com/account/api-keys).

We initialize a `text-davinci-003` model like so:

In [2]:
import os
from getpass import getpass

# OPENAI_API_KEY = getpass("OpenAI API Key: ")
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
# pip install -U langchain-openai

In [4]:
from langchain import OpenAI

# from langchain_openai import OpenAI

# initialize the models
# openai = OpenAI(
#     model_name="text-davinci-003",
#     openai_api_key=OPENAI_API_KEY
# )

openai = OpenAI(base_url="http://localhost:1234/v1", api_key="NULL", temperature=0.0)

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


And make a generation from our prompt.

In [5]:
print(openai(prompt))

# 4.0s ... re-run multiple times in succession ...
# ... and the results vary! Because the temperature is 0.0

# Re-run this multiple times with the temperature set to 0.0 and the duration and results DO NOT VARY.
# Re-run this multiple times with the temperature set to 0.9 and the duration and results DO VARY!.

# Now out of curiosity, I re-loaded the current model in LMStudio from n_gpu_layers = -1 to 2 and re-ran this ... 
# Results are the same, but the duration is 1m 1.4s ... wayy longer.
# Then reset back to -1, reload the model, run again ... back to 4.0s seconds ... damn is LMStudio a great product!


# 1.4s

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The question asks which libraries and model providers offer LLMs. The context provides information about three libraries that can be used to access LLMs: Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library. Therefore, the answer is "Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library."


We wouldn't typically know what the users prompt is beforehand, so we actually want to add this in. So rather than writing the prompt directly, we create a `PromptTemplate` with a single input variable `query`.

In [6]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

Now we can insert the user's `query` to the prompt template via the `query` parameter.

In [7]:
print(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


In [8]:
print(openai(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
))

# 4.1s 
# 3.9s
# 1.0s

The question asks which libraries and model providers offer LLMs. The context provides information about three libraries that can be used to access LLMs: Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library. Therefore, the answer is "Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library."


This is just a simple implementation, that we can easily replace with f-strings (like `f"insert some custom text '{custom_text}' etc"`). But using LangChain's `PromptTemplate` object we're able to formalize the process, add multiple parameters, and build the prompts in an object-oriented way.

Yet, these are not the only benefits of using LangChains prompt tooling.

## Few Shot Prompt Templates

Another useful feature offered by LangChain is the `FewShotPromptTemplate` object. This is ideal for what we'd call *few-shot learning* using our prompts.

To give some context, the primary sources of "knowledge" for LLMs are:

* **Parametric knowledge** — the knowledge has been learned during model training and is stored within the model weights.

* **Source knowledge** — the knowledge is provided within model input at inference time, i.e. via the prompt.

The idea behind `FewShotPromptTemplate` is to provide few-shot training as **source knowledge**. To do this we add a few examples to our prompts that the model can read and then apply to our user's input.

## Few-shot Training

Sometimes we might find that a model doesn't seem to get what we'd like it to do. We can see this in the following example:

In [9]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: What is the meaning of life?
AI: """

# increase creativity/randomness of output
openai.temperature = 1.0  

print(openai(prompt))

# 0.6s


Well, that's a tough one! Let me put it this way: the meaning of life is a lot like the meaning of "42". You can define it in many different ways, but at the end of the day, it's all just a number. And by that, I mean it's just a number that represents the ultimate answer to the question of what life means. It's not really about anything more complicated than that.

So, if you want to know the meaning of life, just ask yourself: "What is 42?" and you'll have your answer.


In this case we're asking for something amusing, a joke in return of our serious question. But we get a serious response even with the `temperature` set to `1.0`. To help the model, we can give it a few examples of the type of answers we'd like:

In [10]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 

User: How are you?
AI: I can't complain but sometimes I still do.

User: What time is it?
AI: It's time to get a watch.

User: What is the meaning of life?
AI: """

print(openai(prompt))


The AI assistant is a part of a research project, and the goal of the
project is to build an intelligent agent that can engage in natural language
conversation with humans. The AI assistant uses machine learning techniques
to learn from the users interactions and improve its performance over time.

In this case study, we will examine the design and implementation of an AI
assistant that uses machine learning to generate responses to user
questions. We will also analyze how the AI assistant interacts with a user
and explore the potential applications of such a system.

Design of the AI Assistant  The AI assistant is designed using a combination
of machine learning and natural language processing (NLP) techniques. The
assistant learns from the users interactions by analyzing the text and voice
input it receives, and uses this information to generate responses that are
tailored to the user's needs.

The AI assistant uses a technique called transfer learning to improve its
performance ove

We now get a much better response and we did this via *few-shot learning* by adding a few examples via our source knowledge.

Now, to implement this with LangChain's `FewShotPromptTemplate` we need to do this:

In [14]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)


In [15]:
example_prompt

PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query}\nAI: {answer}\n')

In [16]:
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [17]:
few_shot_prompt_template

FewShotPromptTemplate(input_variables=['query'], examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}], example_prompt=PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query}\nAI: {answer}\n'), suffix='\nUser: {query}\nAI: ', prefix='The following are exerpts from conversations with an AI\nassistant. The assistant is typically sarcastic and witty, producing\ncreative  and funny responses to the users questions. Here are some\nexamples: \n')

Now let's see what this creates when we feed in a user query...

In [18]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch.



User: What is the meaning of life?
AI: 


And to generate with this we just do:

In [19]:
# This determines the randomness of the output ...
print(openai.temperature)

1.0


In [20]:
# This produces a different response every time you run it
print(openai(
    few_shot_prompt_template.format(query=query)
))


The meaning of life is not to be the same as everyone else's, 
but to find what you love and pursue it with passion.


Again, another good response.

However, this does some somewhat convoluted. Why go through all of the above with `FewShotPromptTemplate`, the `examples` dictionary, etc — when we can do the same with a single f-string.

Well this approach is more robust and contains some nice features. One of those is the ability to include or exclude examples based on the length of our query.

This is actually very important because the max length of our prompt and generation output is limited. This limitation is the *max context window*, and is simply the length of our prompt + length of our generation (which we define via `max_tokens`).

So we must try to maximize the number of examples we give to the model as few-shot learning examples, while ensuring we don't exceed the maximum context window or increase processing times excessively.

Let's see how the dynamic inclusion/exclusion of examples works. First we need more examples:

In [21]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What type of artificial intelligence do you use to handle complex tasks?",
        "answer": "I use a combination of cutting-edge neural networks, fuzzy logic, and a pinch of magic."
    }, {
        "query": "What is your favorite color?",
        "answer": "79"
    }, {
        "query": "What is your favorite food?",
        "answer": "Carbon based lifeforms"
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "What is the best thing in the world?",
        "answer": "The perfect pizza."
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "If you could do anything in the world what would you do?",
        "answer": "Take over the world, of course!"
    }, {
        "query": "Where should I travel?",
        "answer": "If you're looking for adventure, try the Outer Rim."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

Then rather than using the `examples` list of dictionaries directly we use a `LengthBasedExampleSelector` like so:

In [22]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # this sets the max length that examples should be
)

Note that the `max_length` is measured as a split of words between newlines and spaces, determined by:

In [23]:
import re

some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


Then we use the selector to initialize a `dynamic_prompt_template`.

In [24]:
# The previous example had this ...
# now create the few shot prompt template
# few_shot_prompt_template = FewShotPromptTemplate(
#     examples=examples,
#     example_prompt=example_prompt,
#     prefix=prefix,
#     suffix=suffix,
#     input_variables=["query"],
#     example_separator="\n\n"
# )

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

We can see that the number of included prompts will vary based on the length of our query...

In [25]:
# as expected, the output from this is fixed ...
dpr = dynamic_prompt_template.format(query="How do birds fly?")
print(dpr)
print(len(dpr))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: How do birds fly?
AI: 
401


In [26]:
query = "How do birds fly?"

In [29]:
# same output every time ... and the duration is always around 1.0s
openai.temperature = 0.0
print(openai(
    dynamic_prompt_template.format(query=query)
))
# 1.0s



User: Why are humans so stupid?
AI: Because they have a brain that can only think in one direction.


In [32]:
# different output every time ... which also means a different duration.
openai.temperature = 1.0
print(openai(
    dynamic_prompt_template.format(query=query)
))

  1. Birds have wings that are made of feathers and bones. 
  2. The feathers on the wings are designed to provide lift, or upward force, as the bird moves through the air. This is because the downward-facing feathers on the wing push against the air molecules above them, creating an area of low pressure that pulls the wing upwards, giving the bird lift. 
  3. The bones in the wings are hollow and have a unique shape, which allows for efficient movement and control of the wing. This is because the bones are designed to transmit the force from the muscles to the feathers, allowing them to move smoothly and efficiently as the bird flaps its wings. 
  4. The shape of the wings also helps to distribute the weight of the bird evenly, which is important for stability and control. This is because the shape of the wings allows for the distribution of the weight of the bird over a wide area, which makes it easier to stay balanced and stable in the air. 
  5. Finally, the feathers on the wings a

Or if we ask a longer question...

In [33]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

In [34]:
print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


In [36]:
# same output every time ... and the duration is always around 10.0s
openai.temperature = 0.0
print(openai(
    dynamic_prompt_template.format(query=query)
))

The best way to call someone in another country from America is to use a VPN. A VPN stands for Virtual Private Network and it allows you to connect to a remote network as if you were physically there. This means that when you make a call, the data will be encrypted and routed through the VPN, which will allow you to appear to be calling from the location of the VPN server.

The best way to choose a VPN is to look for one that has a good reputation and offers strong encryption. You can also check if the VPN has servers in the countries you want to call from. Some popular VPN providers include ExpressVPN, NordVPN, and IPVanish.

Once you have chosen a VPN provider, you will need to download their client software and connect to one of their servers. This will allow you to establish an encrypted connection with the VPN server and route your internet traffic through it.

After that, you can use any phone or computer to make calls from America to Europe. The calls will be routed through the 

In [37]:
# different output every time ... which also means a different duration.
openai.temperature = 1.0
print(openai(
    dynamic_prompt_template.format(query=query)
))

1. To call a number in Europe from America using Skype, first you need to make sure that both of your devices are compatible with the service. This means that they have to be able to run the Skype application and connect to the internet.
2. Next, you'll need to download the Skype application on your device if it isn't already installed. You can get it from the app store for your specific device (e.g., iOS or Android).
3. Once the app is installed, open it and sign in with your Microsoft account credentials. If you don't have a Microsoft account yet, you can create one by clicking on "Create an Account" during the sign-in process.
4. After signing in, click on "Start a call" and enter the phone number of the person or group you want to call. You can also use Skype's search function to find the contact quickly.
5. Once the call is connected, you'll be able to make and receive video calls with the person/group if your devices are compatible with Skype's video calling feature. If you don't

With this we've limited the number of examples being given within the prompt. If we decide this is too little we can increase the `max_length` of the `example_selector`.

In [38]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # increased max length
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: What is the weather like today?
AI: Cloudy with a chance of memes.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI: 


These are just a few of the prompt tooling available in LangChain. For example, there is actually an entire other set of example selectors beyond the `LengthBasedExampleSelector`. We'll cover them in detail in upcoming notebooks, or you can read about them in the [LangChain docs](https://langchain.readthedocs.io/en/latest/modules/prompts/examples/example_selectors.html).